In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Dropout, MaxPool1D, ReLU, Flatten

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# RESNET IMPORTS
import time

import matplotlib
matplotlib.use('agg')

In [2]:
samples_per_class = 5000
no_of_classes = 2

In [ ]:
# ✅ Update directory path to the correct location
directory = ""

# ✅ Load the datasets from the correct folder
noise_df = pd.read_csv(directory + "Final_Merged_Noise_Reduced_No_Abs.csv", header=None)
noise = noise_df.values.astype(float)

data_BBH_df = pd.read_csv(directory + "Final_BBH_Merged_Noise_Signal_Reduced_No_ABS.csv", header=None)
data_BBH = data_BBH_df.values.astype(float)

In [4]:
X = np.concatenate((noise, data_BBH), axis=0)

print(len(noise_df.index))
print(X)
print(X.shape)

5000
[[-1.48119532e-19 -8.33479173e-20  9.26525534e-21 ... -2.21277986e-19
  -7.00055009e-20 -9.30532001e-20]
 [-7.70934349e-20 -6.68364011e-20  1.34495646e-19 ... -2.28088244e-20
   4.74853440e-20 -1.80698277e-19]
 [-4.74484716e-20  7.68717993e-21  6.15634531e-20 ... -3.35633835e-20
   6.61490799e-21 -2.12534526e-20]
 ...
 [-1.21590442e-19 -1.30634798e-19  8.65398926e-20 ... -1.42406186e-19
  -8.45910130e-20  1.33396622e-19]
 [ 3.51137387e-20 -1.56086508e-20 -2.78166290e-19 ... -2.14004104e-20
   3.35278081e-19  2.90133943e-19]
 [-2.45040393e-19 -1.46934266e-19 -2.05024668e-19 ... -4.64327319e-20
  -3.51327377e-20 -5.89807043e-20]]
(10000, 16384)


In [5]:

# Alternate way of creating y for the dataset
 
y = [int(i/samples_per_class) for i in range(samples_per_class*no_of_classes)]
y = tf.keras.utils.to_categorical(y)
print(y)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [6]:
X *= 1e19
print(X)

# X *= 1e18
# print(X)

[[-1.48119532 -0.83347917  0.09265255 ... -2.21277986 -0.70005501
  -0.930532  ]
 [-0.77093435 -0.66836401  1.34495646 ... -0.22808824  0.47485344
  -1.80698277]
 [-0.47448472  0.0768718   0.61563453 ... -0.33563383  0.06614908
  -0.21253453]
 ...
 [-1.21590442 -1.30634798  0.86539893 ... -1.42406186 -0.84591013
   1.33396622]
 [ 0.35113739 -0.15608651 -2.7816629  ... -0.2140041   3.35278081
   2.90133943]
 [-2.45040393 -1.46934266 -2.05024668 ... -0.46432732 -0.35132738
  -0.58980704]]


In [7]:
X = np.expand_dims(X, axis=-1)
print(X.shape)
print(X)

(10000, 16384, 1)
[[[-1.48119532]
  [-0.83347917]
  [ 0.09265255]
  ...
  [-2.21277986]
  [-0.70005501]
  [-0.930532  ]]

 [[-0.77093435]
  [-0.66836401]
  [ 1.34495646]
  ...
  [-0.22808824]
  [ 0.47485344]
  [-1.80698277]]

 [[-0.47448472]
  [ 0.0768718 ]
  [ 0.61563453]
  ...
  [-0.33563383]
  [ 0.06614908]
  [-0.21253453]]

 ...

 [[-1.21590442]
  [-1.30634798]
  [ 0.86539893]
  ...
  [-1.42406186]
  [-0.84591013]
  [ 1.33396622]]

 [[ 0.35113739]
  [-0.15608651]
  [-2.7816629 ]
  ...
  [-0.2140041 ]
  [ 3.35278081]
  [ 2.90133943]]

 [[-2.45040393]
  [-1.46934266]
  [-2.05024668]
  ...
  [-0.46432732]
  [-0.35132738]
  [-0.58980704]]]


In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [9]:
print(X_train.shape, y_train.shape)

(8000, 16384, 1) (8000, 2)


In [10]:
# resnet model 
# when tuning start with learning rate->mini_batch_size -> 
# momentum-> #hidden_units -> # learning_rate_decay -> #layers 

class Classifier_RESNET:

    def __init__(self, output_directory, input_shape, nb_classes, verbose=False, build=True, load_weights=False):
        self.output_directory = output_directory
        if build == True:
            self.model = self.build_model(input_shape, nb_classes)
            if (verbose == True):
                self.model.summary()
            self.verbose = verbose
            if load_weights == True:
                self.model.load_weights(self.output_directory
                                        .replace('resnet_augment', 'resnet')
                                        .replace('TSC_itr_augment_x_10', 'TSC_itr_10')
                                        + '/model_init.hdf5')
            else:
                self.model.save_weights(self.output_directory + 'model_init.weights.h5')
        return

    def build_model(self, input_shape, nb_classes):
        n_feature_maps = 64

        input_layer = keras.layers.Input(input_shape)

        # BLOCK 1

        conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=5, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # expand channels for the sum
        shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        output_block_1 = keras.layers.add([shortcut_y, conv_z])
        output_block_1 = keras.layers.Activation('relu')(output_block_1)

        # BLOCK 2

        conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_1)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # expand channels for the sum
        shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        output_block_2 = keras.layers.add([shortcut_y, conv_z])
        output_block_2 = keras.layers.Activation('relu')(output_block_2)

        # BLOCK 3

        conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_2)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # no need to expand channels because they are equal
        shortcut_y = keras.layers.BatchNormalization()(output_block_2)

        output_block_3 = keras.layers.add([shortcut_y, conv_z])
        output_block_3 = keras.layers.Activation('relu')(output_block_3)

        # FINAL

        gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)

        output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(),
                      metrics=['accuracy'])

        return model

    def fit(self, x_train, y_train, x_val, y_val, BATCH_SIZE = 64, nb_epochs = 5):
        if not tf.test.is_gpu_available:
            print('error')
            exit()
        # x_val and y_val are only used to monitor the test loss and NOT for training

        start_time = time.time()

        history = self.model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=nb_epochs, validation_data=(x_val, y_val))

        keras.backend.clear_session()


        return history


In [11]:
RESNET = Classifier_RESNET(directory, (16384, 1), 2, verbose = True)

2025-03-18 22:50:56.861958: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2025-03-18 22:50:56.861977: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2025-03-18 22:50:56.861981: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2025-03-18 22:50:56.861997: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-18 22:50:56.862007: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 16384, 1)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 16384, 64) │        576 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 16384, 64) │        256 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 16384, 64) │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 16384, 64) │     20,544 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16384, 64) │        256 │ conv1d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 16384, 64) │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 16384, 64) │        128 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 16384, 64) │     12,352 │ activation_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16384, 64) │        256 │ conv1d_3[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16384, 64) │        256 │ conv1d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 16384, 64) │          0 │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 16384, 64) │          0 │ add[0][0]         │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 16384,     │     65,664 │ activation_2[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16384,     │        512 │ conv1d_4[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 16384,     │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 16384,     │     82,048 │ activation_3[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16384,     │        512 │ conv1d_5[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                 

 Total params: 506,818 (1.93 MB)

 Trainable params: 504,258 (1.92 MB)

 Non-trainable params: 2,560 (10.00 KB)

In [ ]:
BATCH_SIZE = 128
EPOCHS = 10

history = RESNET.fit(X_train, y_train, X_val, y_val, BATCH_SIZE, EPOCHS)

Epoch 1/10


2025-03-18 22:50:58.546446: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


63/63 ━━━━━━━━━━━━━━━━━━━━ 585s 9s/step - accuracy: 0.9508 - loss: 0.2224 - val_accuracy: 0.9710 - val_loss: 0.2448
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 570s 9s/step - accuracy: 0.9999 - loss: 0.0314 - val_accuracy: 0.9935 - val_loss: 0.2041
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 603s 10s/step - accuracy: 0.9988 - loss: 0.0191 - val_accuracy: 0.9790 - val_loss: 0.1075
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 605s 10s/step - accuracy: 0.9996 - loss: 0.0096 - val_accuracy: 0.9870 - val_loss: 0.0784
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 594s 9s/step - accuracy: 1.0000 - loss: 0.0066 - val_accuracy: 0.9975 - val_loss: 0.0226
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 599s 10s/step - accuracy: 1.0000 - loss: 0.0044 - val_accuracy: 0.9980 - val_loss: 0.0089
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 3770s 61s/step - accuracy: 1.0000 - loss: 0.0038 - val_accuracy: 0.9975 - val_loss: 0.0515
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4705s 76s/step - accuracy: 1.0000 - loss: 0.0026 - val_accuracy: 0.9990 - val_l

: 

In [ ]:
import matplotlib
matplotlib.use('TkAgg')  # Ensure interactive plotting

import matplotlib.pyplot as plt

# Extract accuracy and loss
acc = history.history['accuracy']
val_acc = history.history.get('val_accuracy', None)  # Check if validation accuracy exists
loss = history.history['loss']
val_loss = history.history.get('val_loss', None)  # Check if validation loss exists

plt.figure(figsize=(13, 6))

# ✅ Plot Training & Validation Accuracy
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy', marker='o')
if val_acc is not None:
    plt.plot(val_acc, label='Validation Accuracy', marker='x', linestyle='dashed')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(acc) * 0.95, 1])
plt.title('Training & Validation Accuracy')
plt.xlabel('Epoch')

# ✅ Plot Training & Validation Loss
plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss', marker='o', color='blue')
if val_loss is not None:
    plt.plot(val_loss, label='Validation Loss', marker='x', linestyle='dashed', color='red')
plt.legend(loc='upper right')
plt.ylabel('Categorical Cross Entropy')
plt.ylim([0, max(loss) * 1.1])
plt.title('Training & Validation Loss')
plt.xlabel('Epoch')

# ✅ Show the plots
plt.tight_layout()
plt.show()

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

# Get model predictions
pred = RESNET.model.predict(X)
y_pred = np.argmax(pred, axis=1)
y_true = np.argmax(y, axis=1)

# Compute precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='binary')
print('Precision: %.3f' % precision)

recall = recall_score(y_true, y_pred, average='binary')
print('Recall: %.3f' % recall)

score = f1_score(y_true, y_pred, average='binary')
print('F1-Score: %.3f' % score)

# Compute and display confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot()

# ✅ Fix: Handle non-interactive environments
try:
    plt.show()
except:
    plt.savefig("confusion_matrix.png")  # Saves the confusion matrix if plt.show() fails
    print("✅ Confusion Matrix saved as 'confusion_matrix.png'")

NameError: name 'RESNET' is not defined